# Value-at-risk calculations

_Do not use this code to guide actual investment decisions!_

## Basic setup

In [ ]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.getOrCreate()

## Loading the data

In [ ]:
df = spark.read.load("/data/wikieod.parquet").cache()

## Calculating historical returns

We'll use Spark's windowing functions over data frames to determine the percentage change in each security's price from the previous close to each day's close.

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col, avg, variance

ddf = df.select("ticker", "date", "close").withColumn("change", (col("close") / lag("close", 1).over(Window.partitionBy("ticker").orderBy(df["date"])) - 1.0) * 100)
ddf.show(10)

## Characterizing expected return distributions

In [ ]:
from pyspark.sql.functions import sqrt
mv = ddf.groupBy("ticker").agg(avg("change").alias("mean"), sqrt(variance("change")).alias("stddev"))
mv.show(10)

Since there are only about 3,000 ticker symbols in our data set, we can easily collect these in driver memory for use in our simulation:

In [ ]:
dist_map = mv.rdd.map(lambda r: (r[0], (r[1], r[2]))).collectAsMap()
dist_map["RHT"]

## Getting current security prices

In [ ]:
from pyspark.sql.functions import first
priceDF = ddf.orderBy("date", ascending=False).groupBy("ticker").agg(first("close").alias("price"), first("date").alias("date")).cache()
priceDF.show(10)

prices = priceDF.rdd.map(lambda r: (r[0], r[1])).collectAsMap()

## Setting up a simulation

### Generating a random portfolio

In [ ]:
from random import randint, seed

def random_portfolio(symbols):
    result = {}
    for s in symbols:
        result[s] = prices[s] * (randint(1, 1000) * 11)
    return result

def portfolio_value(pf):
    return sum([v for v in pf.values()])

seed(0xdea110c8)

portfolio = random_portfolio(ddf.select("ticker").distinct().sample(True, 0.01, 0xdea110c8).rdd.map(lambda r: r[0]).collect())

### Generating random seeds for each simulation

In [ ]:
def seeds(count):
    return [randint(0, 1 << 32 - 1) for i in range(count)]

### Defining the simulation

In [ ]:
def simstep(pf, params, prng):
    def daily_return(sym):
        mean, stddev = params[sym]
        change = (prng.normalvariate(mean, stddev) + 100) / 100.0
        return change
    return dict([(s, daily_return(s) * v) for s, v in pf.items()])

def simulate(seed, pf, params, days):
    from random import Random
    prng = Random()
    prng.seed(seed)
    pf = pf.copy()
    
    for day in range(days):
        pf = simstep(pf, params, prng)
    return pf

### Simulating five days of market activity

In [ ]:
days_to_simulate = 5
simulation_count = 10000

sc = spark.sparkContext
seed_rdd = sc.parallelize(seeds(simulation_count))
bparams = sc.broadcast(dist_map)
bpf = sc.broadcast(portfolio)
initial_value = portfolio_value(portfolio)

results = seed_rdd.map(lambda s: portfolio_value(simulate(s, bpf.value, bparams.value, days_to_simulate)) - initial_value)

In [ ]:
simulated_results = list(zip(results.collect(), seed_rdd.collect()))
simulated_values = [v for (v, _) in simulated_results]
simulated_values.sort()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import seaborn as sns
import numpy as np
sns.set(color_codes=True)

_ = sns.distplot(simulated_values, kde=False).set(xlabel="$ change", ylabel="simulation count")


In [ ]:
# plot only every 20th simulation result (unless that would leave us with fewer 
# than 50 elements) to avoid generating a huge figure
plotevery = len(simulated_values) > 1000 and 20 or 1

xvals = [float(i) / len(simulated_values) for i in range(len(simulated_values))]
ax = sns.tsplot(np.array(simulated_values[::plotevery]), np.array(xvals[::plotevery]))

# add ticks for every ten percent
ax.get_xaxis().set_ticks([i * 0.1 for i in range(11)])
_ = ax.get_xaxis().set_ticklabels(["%d%%" % (i * 10) for i in range(11)])

_ = ax.set(xlabel="cumulative percentage of simulations with at least this change", ylabel="gain or loss")

### Identifying the 5% value-at-risk

In [ ]:
percentage_var = 0.05

simulated_values[int(len(simulated_values) * percentage_var)]

## Visualizing random walks by retaining history

In [ ]:
def simulate_with_history(seed, pf, params, days):
    from random import Random
    prng = Random()
    prng.seed(seed)
    pf = pf.copy()
    values = [portfolio_value(pf)]
    
    for day in range(days):
        pf = simstep(pf, params, prng)
        values.append(portfolio_value(pf))
    return values

### Collecting results at each decile

In [ ]:
simulated_results.sort()

eleven_results = [simulated_results[int((len(simulated_results) - 1) * i / 10)] for i in range(11)]
eleven_seeds = sc.parallelize([seed for (_, seed) in eleven_results])
walks = eleven_seeds.map(lambda s: simulate_with_history(s, bpf.value, bparams.value, 5))

walk_results = walks.collect()

In [ ]:
for c in walk_results:
    ax = sns.tsplot(c)

_ = ax.set(xlabel="day of simulation", ylabel="total portfolio value")

# Getting more realistic results

Of course, most real-world stock returns aren't normally distributed.  To make a more interesting simulation, we can try to find a distribution that better models the returns we've observed.  We'll start by looking at the actual distributions of returns.

In [ ]:
rdist = ddf.filter(ddf["ticker"] == "RHT").select("change").rdd.map(lambda r: r["change"]).filter(lambda c: c is not None).collect()
sns.distplot(rdist)

## Stock price changes aren't normally-distributed

In [ ]:
from scipy import stats
sns.distplot(rdist, fit=stats.norm)

## Trying different distributions

In [ ]:
symbols = ddf.select("ticker").distinct().sample(True, 0.004).rdd.map(lambda l: l["ticker"]).collect()
dfs = ddf.filter(ddf["ticker"].isin(symbols)).select("ticker", "change").dropna()
sampled_returns = dfs.toPandas()

In [ ]:
g = sns.FacetGrid(sampled_returns, row="ticker", sharex=False, sharey=False, aspect=3)
_ = g.map(sns.distplot, "change", fit=stats.norm)

In [ ]:
g2 = sns.FacetGrid(sampled_returns, row="ticker", sharex=False, sharey=False, aspect=3)
_ = g2.map(sns.distplot, "change", fit=stats.t)